In [313]:
import numpy as np
import pandas as pd
import itertools
from itertools import combinations
import calendar
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import TimeSeriesSplit
from collections import Counter
import math
from sklearn.model_selection import PredefinedSplit

In [314]:
#momentum factor (monthly)
ff = pd.read_csv("F-F_Momentum_Factor.csv")

#monthly returns on VWM
vwm = pd.read_csv("VWM Monthly Returns.csv")

# the 1, 5, and 10-year constant maturity series from FRED monthly
one = pd.read_csv("GS1.csv")
five = pd.read_csv("GS5.csv")
ten = pd.read_csv("GS10.csv")

#AAA and BAA monthly
AAA = pd.read_csv("AAA.csv")
BAA = pd.read_csv("BAA.csv")

#CPI monthly
CPI = pd.read_csv("CPILFESL.csv")

#Unemployment rate monthly
UR = pd.read_csv("UNRATE.csv")

#Industrial Productivity monthly
INDPRO = pd.read_csv("INDPRO.csv")

# Display the first few rows of the DataFrame
print(ff.head())
print(vwm.head())
print(one.head())
print(five.head())
print(ten.head())
print(AAA.head())
print(BAA.head())
print(CPI.head())
print(UR.head())
print(INDPRO.head())




   Unnamed: 0  Mom   
0      195701   -3.09
1      195702    0.32
2      195703   -0.18
3      195704    1.00
4      195705    0.48
   Unnamed: 0   <= 0  Lo 30  Med 40  Hi 30  Lo 20  Qnt 2  Qnt 3  Qnt 4  Hi 20  \
0      195701 -99.99   2.12   -0.43  -3.80   2.37   0.97  -0.11  -1.00  -4.04   
1      195702 -99.99  -2.42   -2.32  -1.77  -2.03  -2.51  -2.72  -1.81  -1.77   
2      195703 -99.99   1.66    2.66   2.33   1.65   1.95   2.42   2.54   2.34   
3      195704 -99.99   2.32    2.31   4.85   2.69   2.57   2.10   2.98   4.98   
4      195705 -99.99   1.32    2.40   3.93   0.37   2.01   2.52   2.22   4.09   

   Lo 10  02-Dec  03-Dec  04-Dec  05-Dec  06-Dec  07-Dec  08-Dec  09-Dec  \
0   4.63    1.39    1.90    0.34   -0.30    0.02   -1.02   -0.99   -1.61   
1  -2.89   -1.64   -2.78   -2.32   -3.45   -2.22   -1.87   -1.77   -1.82   
2   0.67    2.09    1.67    2.14    2.85    2.14    3.08    2.20    2.41   
3   3.50    2.34    1.98    2.98    1.97    2.19    2.34    3.38    3.14   
4

In [315]:
#Variable Construction

#TERM - The difference between the 10-year and the 1-year (Y10-Y1)

TERM = ten['GS10']-one['GS1']

#CURVE - The 10-year yield plus the 1-year yield minus 2 times the 5-year yield Y10 −2Y5 +Y1 = (Y10 −Y5)−(Y5 −Y1).

CURVE = ten['GS10'] + one['GS1'] - 2*five['GS5']

# DEFAULT - The difference between the AAA and the BAA yields YAAA-YBAA
DEFAULT = AAA['AAA']-BAA['BAA']



In [316]:
CPI.set_index('observation_date', inplace=True)


In [317]:
# INFLATION - The year-over-year difference of log CPI: lnCPIt −lnCPIt−12.
# Step 1: Compute log(CPI)
CPI['log_CPI'] = np.log(CPI['CPILFESL'])
# Step 2: Shift log(CPI) by 12 months
CPI['log_CPI_lag_12'] = CPI['log_CPI'].shift(12)
# Step 3: Compute year-over-year difference
CPI['INFLATION'] = CPI['log_CPI'] - CPI['log_CPI_lag_12']
print(CPI)

                  CPILFESL   log_CPI  log_CPI_lag_12  INFLATION
observation_date                                               
1957-01-01          28.500  3.349904             NaN        NaN
1957-02-01          28.600  3.353407             NaN        NaN
1957-03-01          28.700  3.356897             NaN        NaN
1957-04-01          28.800  3.360375             NaN        NaN
1957-05-01          28.800  3.360375             NaN        NaN
...                    ...       ...             ...        ...
2024-06-01         318.346  5.763139        5.730895   0.032244
2024-07-01         318.872  5.764790        5.733163   0.031627
2024-08-01         319.768  5.767596        5.735462   0.032134
2024-09-01         320.767  5.770715        5.738648   0.032067
2024-10-01         321.666  5.773514        5.741046   0.032468

[814 rows x 4 columns]


In [318]:
CPI.reset_index(inplace=True)
 

In [319]:
CPI

,observation_date,CPILFESL,log_CPI,log_CPI_lag_12,INFLATION
0,1957-01-01,28.500,3.349904,NaN,NaN
1,1957-02-01,28.600,3.353407,NaN,NaN
2,1957-03-01,28.700,3.356897,NaN,NaN
3,1957-04-01,28.800,3.360375,NaN,NaN
4,1957-05-01,28.800,3.360375,NaN,NaN
...,...,...,...,...,...
809,2024-06-01,318.346,5.763139,5.730895,0.032244
810,2024-07-01,318.872,5.764790,5.733163,0.031627
811,2024-08-01,319.768,5.767596,5.735462,0.032134
812,2024-09-01,320.767,5.770715,5.738648,0.032067


In [320]:
INFLATION = CPI['INFLATION']

In [321]:
print(INFLATION.head(13))


0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
5          NaN
6          NaN
7          NaN
8          NaN
9          NaN
10         NaN
11         NaN
12    0.027683
Name: INFLATION, dtype: float64


ANALYSIS

In [322]:
#number of rows of each dataframe
INFLATION.shape




(814,)

In [323]:
#Split the data into in sample and out of sample

#have to start from 12 because INFLATION has no numbers for the first 12 columns

#814-12 = 802, 802/2 = 401 obs in each split of data, so start from 12 go to 12+401 = 

#in sample _train, out of sample _test
vwm_train = vwm[12:413]
vwm_test = vwm[413:]

ff_train = ff[12:413]
ff_test = ff[413:]

TERM_train = TERM[12:413]
TERM_test = TERM[413:]

CURVE_train = CURVE[12:413]
CURVE_test = CURVE[413:]

DEFAULT_train = DEFAULT[12:413]
DEFAULT_test = DEFAULT[413:]

INFLATION_train = INFLATION[12:413]
INFLATION_test = INFLATION[413:]



In [324]:
vwm_train

,Unnamed: 0,<= 0,Lo 30,Med 40,Hi 30,Lo 20,Qnt 2,Qnt 3,Qnt 4,Hi 20,Lo 10,02-Dec,03-Dec,04-Dec,05-Dec,06-Dec,07-Dec,08-Dec,09-Dec,Hi 10
12,195801,-99.99,12.02,9.71,4.28,13.09,11.34,10.08,8.68,3.93,13.77,12.80,11.03,11.56,10.77,9.63,8.83,8.59,5.73,3.57
13,195802,-99.99,-1.02,-0.44,-1.56,-1.58,0.17,-0.44,-0.75,-1.63,-2.13,-1.34,-0.49,0.63,-1.32,0.14,-0.73,-0.77,-0.38,-1.87
14,195803,-99.99,3.78,3.77,3.30,3.58,3.24,3.71,3.69,3.29,3.49,3.61,3.96,2.76,3.97,3.54,4.13,3.42,3.04,3.34
15,195804,-99.99,3.76,2.43,3.27,3.46,3.42,2.07,3.28,3.23,5.03,2.75,4.02,3.00,2.33,1.89,2.68,3.65,3.78,3.12
16,195805,-99.99,3.90,4.06,2.17,4.05,3.95,4.77,2.80,2.15,6.16,3.08,3.75,4.09,5.14,4.53,3.27,2.51,3.24,1.93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
408,199101,-99.99,7.99,7.37,4.65,7.84,7.95,7.92,6.17,4.49,7.34,8.35,8.19,7.77,9.26,6.98,6.54,5.93,6.26,4.04
409,199102,-99.99,11.80,9.90,7.08,12.78,11.21,9.28,8.64,6.99,12.89,12.66,10.53,11.73,10.06,8.71,9.87,7.86,8.20,6.67
410,199103,-99.99,7.51,4.51,2.59,8.16,6.20,4.88,4.03,2.37,9.57,6.74,6.64,5.87,5.25,4.62,3.46,4.39,2.96,2.22
411,199104,-99.99,0.79,0.25,0.25,1.25,0.33,0.37,0.74,0.13,2.66,-0.19,0.17,0.45,1.07,-0.15,0.02,1.21,1.64,-0.26


In [325]:
vwm_test

,Unnamed: 0,<= 0,Lo 30,Med 40,Hi 30,Lo 20,Qnt 2,Qnt 3,Qnt 4,Hi 20,Lo 10,02-Dec,03-Dec,04-Dec,05-Dec,06-Dec,07-Dec,08-Dec,09-Dec,Hi 10
413,199106,-99.99,-4.36,-4.05,-4.60,-4.52,-4.33,-4.01,-4.61,-4.54,-4.01,-5.06,-4.14,-4.48,-3.59,-4.33,-3.89,-5.07,-5.71,-4.22
414,199107,-99.99,3.42,4.94,4.76,3.31,3.86,5.19,4.65,4.79,2.16,4.22,3.55,4.14,4.95,5.39,4.96,4.46,4.64,4.83
415,199108,-99.99,3.17,3.52,2.60,3.03,3.71,4.40,2.38,2.62,1.88,3.94,3.34,4.02,4.84,4.05,2.15,2.51,2.62,2.61
416,199109,-99.99,1.31,-0.54,-1.39,1.25,0.55,-0.50,-0.66,-1.50,0.80,1.59,1.38,-0.16,-0.26,-0.68,-0.76,-0.60,-0.94,-1.64
417,199110,-99.99,3.32,1.94,1.52,3.90,2.35,2.44,1.84,1.43,2.36,5.07,2.58,2.15,2.67,2.26,1.17,2.23,2.44,1.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
809,202406,-99.99,-2.78,-1.27,3.64,-3.17,-2.03,-0.84,-1.49,3.97,-5.08,-1.96,-2.36,-1.85,-0.78,-0.89,-1.55,-1.46,-0.64,4.60
810,202407,-99.99,11.75,7.69,1.05,12.24,10.70,8.07,5.59,0.79,9.95,13.80,11.21,10.39,7.83,8.23,6.08,5.33,3.47,0.43
811,202408,-99.99,-3.17,-0.23,2.39,-2.92,-1.48,-1.05,1.50,2.42,-2.93,-2.92,-3.45,-0.31,-1.22,-0.93,0.84,1.84,1.77,2.51
812,202409,-99.99,-0.02,1.81,2.21,-0.45,0.34,1.89,2.27,2.21,0.48,-1.05,0.46,0.27,2.25,1.64,2.38,2.21,2.41,2.19


KITCHEN SINK MODEL

FOR VWM 

In [326]:
import statsmodels.api as sm
from IPython.display import HTML

In [327]:
#each column represents the size of the portfolio, lo30 means lowest 30% in size, lets just use 05-Dec for now
vwm_test

,Unnamed: 0,<= 0,Lo 30,Med 40,Hi 30,Lo 20,Qnt 2,Qnt 3,Qnt 4,Hi 20,Lo 10,02-Dec,03-Dec,04-Dec,05-Dec,06-Dec,07-Dec,08-Dec,09-Dec,Hi 10
413,199106,-99.99,-4.36,-4.05,-4.60,-4.52,-4.33,-4.01,-4.61,-4.54,-4.01,-5.06,-4.14,-4.48,-3.59,-4.33,-3.89,-5.07,-5.71,-4.22
414,199107,-99.99,3.42,4.94,4.76,3.31,3.86,5.19,4.65,4.79,2.16,4.22,3.55,4.14,4.95,5.39,4.96,4.46,4.64,4.83
415,199108,-99.99,3.17,3.52,2.60,3.03,3.71,4.40,2.38,2.62,1.88,3.94,3.34,4.02,4.84,4.05,2.15,2.51,2.62,2.61
416,199109,-99.99,1.31,-0.54,-1.39,1.25,0.55,-0.50,-0.66,-1.50,0.80,1.59,1.38,-0.16,-0.26,-0.68,-0.76,-0.60,-0.94,-1.64
417,199110,-99.99,3.32,1.94,1.52,3.90,2.35,2.44,1.84,1.43,2.36,5.07,2.58,2.15,2.67,2.26,1.17,2.23,2.44,1.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
809,202406,-99.99,-2.78,-1.27,3.64,-3.17,-2.03,-0.84,-1.49,3.97,-5.08,-1.96,-2.36,-1.85,-0.78,-0.89,-1.55,-1.46,-0.64,4.60
810,202407,-99.99,11.75,7.69,1.05,12.24,10.70,8.07,5.59,0.79,9.95,13.80,11.21,10.39,7.83,8.23,6.08,5.33,3.47,0.43
811,202408,-99.99,-3.17,-0.23,2.39,-2.92,-1.48,-1.05,1.50,2.42,-2.93,-2.92,-3.45,-0.31,-1.22,-0.93,0.84,1.84,1.77,2.51
812,202409,-99.99,-0.02,1.81,2.21,-0.45,0.34,1.89,2.27,2.21,0.48,-1.05,0.46,0.27,2.25,1.64,2.38,2.21,2.41,2.19


In [328]:
INFLATION

0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
         ...   
809    0.032244
810    0.031627
811    0.032134
812    0.032067
813    0.032468
Name: INFLATION, Length: 814, dtype: float64

In [329]:
#construct dataframe containing all regressors
factors = pd.DataFrame()
factors['Term'] = TERM_train
factors['Curve'] = CURVE_train
factors['Default'] = DEFAULT_train
factors['Inflation'] = INFLATION_train

 

In [330]:
factors

,Term,Curve,Default,Inflation
12,0.44,-0.02,-1.23,0.027683
13,1.06,-0.52,-1.07,0.027588
14,1.14,-0.46,-1.05,0.027493
15,1.43,-0.59,-1.07,0.024015
16,1.55,-0.53,-1.05,0.024015
...,...,...,...,...
408,1.45,-0.67,-1.41,0.054505
409,1.58,-0.82,-1.24,0.054979
410,1.71,-1.03,-1.16,0.051106
411,1.80,-1.12,-1.08,0.050211


In [331]:
factors_w_constant = sm.add_constant(factors)
factors_w_constant.head()

,const,Term,Curve,Default,Inflation
12,1.0,0.44,-0.02,-1.23,0.027683
13,1.0,1.06,-0.52,-1.07,0.027588
14,1.0,1.14,-0.46,-1.05,0.027493
15,1.0,1.43,-0.59,-1.07,0.024015
16,1.0,1.55,-0.53,-1.05,0.024015


In [332]:
# Make it more efficient: loop over all portfolios
heter_results_vwm = {}
homosk_results_vwm = {}

for model in vwm_train.columns:
    mod = sm.OLS(vwm_train[model], factors_w_constant)
    heter_results_vwm[model] = mod.fit(cov_type="HC0")
    homosk_results_vwm[model] = sm.OLS(vwm_train[model], factors_w_constant).fit()

In [333]:
heter_results_vwm

{'Unnamed: 0': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x17ce7e090>,
 '<= 0': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x17cdb9760>,
 'Lo 30': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x17cea2630>,
 'Med 40': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x17cea15e0>,
 'Hi 30': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x17cea2f90>,
 'Lo 20': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x17cea1f40>,
 'Qnt 2': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x17cea28d0>,
 'Qnt 3': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x17cea0e90>,
 'Qnt 4': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x17cea19a0>,
 'Hi 20': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x17cea1490>,
 'Lo 10': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x17cea35c0>,
 '02-Dec': <stat

In [334]:
heter_results_vwm["Lo 30"].summary()
# can repeat this for every column in vwm_train

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Lo 30   R-squared:                       0.057
Model:                            OLS   Adj. R-squared:                  0.047
Method:                 Least Squares   F-statistic:                     7.013
Date:                Wed, 08 Jan 2025   Prob (F-statistic):           1.84e-05
Time:                        22:33:10   Log-Likelihood:                -1279.2
No. Observations:                 401   AIC:                             2568.
Df Residuals:                     396   BIC:                             2588.
Df Model:                           4                                         
Covariance Type:                  HC0                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.4472      0.712     -0.628      0.530      -1.843       0.949
Term           2.4083      0.623      3.864      0.000       1.187       3.630
Curve          3.6373      1.115      3.261      0.001       1.451       5.824
Default       -2.4928      0.931     -2.678      0.007      -4.317      -0.668
Inflation    -21.8795     17.113     -1.279      0.201     -55.421      11.662
==============================================================================
Omnibus:                       40.673   Durbin-Watson:                   1.625
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              164.468
Skew:                          -0.313   Prob(JB):                     1.93e-36
Kurtosis:                       6.074   Cond. No.                         102.
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
"""

FOR FF

In [335]:
ff_train

,Unnamed: 0,Mom
12,195801,-7.62
13,195802,3.73
14,195803,-1.88
15,195804,1.84
16,195805,-1.80
...,...,...
408,199101,-6.45
409,199102,-4.61
410,199103,2.87
411,199104,-2.36


In [336]:
heter_results_ff = {}
homosk_results_ff = {}

for model in ff_train.columns:
    mod = sm.OLS(ff_train[model], factors_w_constant)
    heter_results_ff[model] = mod.fit(cov_type="HC0")
    homosk_results_ff[model] = sm.OLS(ff_train[model], factors_w_constant).fit()

In [337]:
heter_results_ff

{'Unnamed: 0': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x17a1463c0>,
 'Mom   ': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x17888e2a0>}

LEFT TO DO
- model selection using either forward or backward selection 
- pick best model for both VWM and ff
- use the best model to predict in sample
- choose return threshold and weights for risky and risk free asset
- construct portfolios with risky and risk free asset based on this weight function
- compute sharpe ratios for these portfolios and compare it to the sharpe ratio of the original portolio??

- repeat step 3 and below for out of sample